# HDB Price vs School Proximity Analysis

Simple analysis showing how school distance affects HDB resale prices using interactive OpenStreetMap visualization.

In [1]:
# Import essential libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import pearsonr

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## Data Loading and Basic Analysis

In [2]:
# Load the dataset
df = pd.read_csv('datasets/train.csv')

print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")

# Check what columns are available
print(f"\nAvailable columns:")
print(df.columns.tolist())

# Check for date-related columns
date_columns = [col for col in df.columns if any(word in col.lower() for word in ['date', 'month', 'year', 'time'])]
if date_columns:
    print(f"\nDate-related columns found: {date_columns}")
    # Try to show date range if available
    for col in date_columns:
        try:
            print(f"{col} range: {df[col].min()} to {df[col].max()}")
        except:
            print(f"{col} - unable to show range")
else:
    print("\nNo obvious date columns found")

# Show key columns for analysis
key_columns = ['town', 'resale_price', 'floor_area_sqm', 'hdb_age', 
               'pri_sch_nearest_distance', 'sec_sch_nearest_distance',
               'Latitude', 'Longitude']

print(f"\nKey columns for analysis:")
for col in key_columns:
    if col in df.columns:
        print(f"✓ {col}")
    else:
        print(f"✗ {col} - missing")

# Basic statistics
print(f"\nPrice Statistics:")
print(f"Average: ${df['resale_price'].mean():,.0f}")
print(f"Median: ${df['resale_price'].median():,.0f}")
print(f"Range: ${df['resale_price'].min():,.0f} - ${df['resale_price'].max():,.0f}")

df.head()

/var/folders/jm/_tt1vmwd5j18rmn0gnz0ktkc0000gn/T/ipykernel_3819/2630000924.py:2: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('datasets/train.csv')


Dataset loaded: 150634 rows, 77 columns

Available columns:
['id', 'Tranc_YearMonth', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price', 'Tranc_Year', 'Tranc_Month', 'mid_storey', 'lower', 'upper', 'mid', 'full_flat_type', 'address', 'floor_area_sqft', 'hdb_age', 'max_floor_lvl', 'year_completed', 'residential', 'commercial', 'market_hawker', 'multistorey_carpark', 'precinct_pavilion', 'total_dwelling_units', '1room_sold', '2room_sold', '3room_sold', '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold', 'studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental', 'other_room_rental', 'postal', 'Latitude', 'Longitude', 'planning_area', 'Mall_Nearest_Distance', 'Mall_Within_500m', 'Mall_Within_1km', 'Mall_Within_2km', 'Hawker_Nearest_Distance', 'Hawker_Within_500m', 'Hawker_Within_1km', 'Hawker_Within_2km', 'hawker_food_stalls', 'hawker_market_stalls', 'mrt_nearest_distance', 'mrt_name', 'bus

,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
0,88471,2016-05,KALLANG/WHAMPOA,4 ROOM,3B,UPP BOON KENG RD,10 TO 12,90.0,Model A,2006,...,78,1,1.317659,103.882504,1138.633422,Geylang Methodist School,224,0,1.317659,103.882504
1,122598,2012-07,BISHAN,5 ROOM,153,BISHAN ST 13,07 TO 09,130.0,Improved,1987,...,45,1,1.349783,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,0,1.350110,103.854892
2,170897,2013-07,BUKIT BATOK,EXECUTIVE,289B,BT BATOK ST 25,13 TO 15,144.0,Apartment,1997,...,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013
3,86070,2012-04,BISHAN,4 ROOM,232,BISHAN ST 22,01 TO 05,103.0,Model A,1992,...,20,1,1.354789,103.844934,389.515528,Catholic High School,253,1,1.354789,103.844934
4,153632,2017-12,YISHUN,4 ROOM,876,YISHUN ST 81,01 TO 03,83.0,Simplified,1987,...,74,0,1.416280,103.838798,312.025435,Orchid Park Secondary School,208,0,1.414888,103.838335


In [3]:
# Clean data for school analysis (using correct column names)
df_clean = df.dropna(subset=['pri_sch_nearest_distance', 'sec_sch_nearest_dist', 
                            'Latitude', 'Longitude', 'resale_price'])

print(f"Cleaned dataset: {df_clean.shape[0]} rows (removed {df.shape[0] - df_clean.shape[0]} rows with missing school data)")

# Calculate correlation between price and school distances
pri_corr, pri_p = pearsonr(df_clean['resale_price'], df_clean['pri_sch_nearest_distance'])
sec_corr, sec_p = pearsonr(df_clean['resale_price'], df_clean['sec_sch_nearest_dist'])

print(f"\nSchool Distance Correlations:")
print(f"Primary school distance vs Price: {pri_corr:.3f} (p={pri_p:.3f})")
print(f"Secondary school distance vs Price: {sec_corr:.3f} (p={sec_p:.3f})")

if pri_corr < -0.1:
    print("Primary schools show negative correlation - closer = higher price!")
if sec_corr < -0.1:
    print("Secondary schools show negative correlation - closer = higher price!")

Cleaned dataset: 150634 rows (removed 0 rows with missing school data)

School Distance Correlations:
Primary school distance vs Price: -0.011 (p=0.000)
Secondary school distance vs Price: 0.095 (p=0.000)


## Interactive OpenStreetMap Visualization

In [4]:
# Sample data for faster visualization (take 2000 random points)
df_map = df_clean.sample(n=min(2000, len(df_clean)), random_state=42)

# Create combined school distance metric (average of primary and secondary)
df_map['avg_school_distance'] = (df_map['pri_sch_nearest_distance'] + df_map['sec_sch_nearest_dist']) / 2

# Create interactive map
fig = px.scatter_mapbox(
    df_map,
    lat="Latitude",
    lon="Longitude",
    color="resale_price",
    size="avg_school_distance",
    color_continuous_scale="Viridis",
    size_max=15,
    zoom=10,
    mapbox_style="open-street-map",
    title="HDB Resale Prices vs School Proximity",
    hover_data={
        'town': True,
        'resale_price': ':$,.0f',
        'pri_sch_nearest_distance': ':.1f',
        'sec_sch_nearest_dist': ':.1f',
        'floor_area_sqm': ':.0f',
        'hdb_age': ':.0f'
    }
)

fig.update_layout(
    height=600,
    coloraxis_colorbar=dict(
        title="Resale Price ($)",
        title_font_size=12
    )
)

print("Interactive Map Legend:")
print("Red/Yellow dots = Higher prices")
print("Green/Blue dots = Lower prices") 
print("Larger circles = Further from schools")
print("Smaller circles = Closer to schools")

fig.show()

Interactive Map Legend:
Red/Yellow dots = Higher prices
Green/Blue dots = Lower prices
Larger circles = Further from schools
Smaller circles = Closer to schools


## Summary

In [6]:
# Key insights from the analysis
print("KEY FINDINGS:")
print("=" * 50)

# School distance insights
if pri_corr < -0.1:
    print(f"SCHOOL PROXIMITY MATTERS:")
    print(f"   • Primary schools: {abs(pri_corr):.1%} negative correlation")
    print(f"   • Closer to primary school = Higher resale price")

if sec_corr < -0.1:
    print(f"   • Secondary schools: {abs(sec_corr):.1%} negative correlation") 
    print(f"   • Closer to secondary school = Higher resale price")

# Price range by distance
close_primary = df_clean[df_clean['pri_sch_nearest_distance'] <= 0.5]
far_primary = df_clean[df_clean['pri_sch_nearest_distance'] > 1.0]

if len(close_primary) > 0 and len(far_primary) > 0:
    price_diff = close_primary['resale_price'].median() - far_primary['resale_price'].median()
    print(f"\nPRICE IMPACT:")
    print(f"   • Close to primary school (≤0.5km): ${close_primary['resale_price'].median():,.0f}")
    print(f"   • Far from primary school (>1.0km): ${far_primary['resale_price'].median():,.0f}")
    print(f"   • Price difference: ${price_diff:,.0f}")

print(f"\nThe interactive map above shows:")
print(f"   • Red/yellow dots = Higher prices")
print(f"   • Green/blue dots = Lower prices")
print(f"   • Large circles = Further from schools")
print(f"   • Small circles = Closer to schools")

KEY FINDINGS:

The interactive map above shows:
   • Red/yellow dots = Higher prices
   • Green/blue dots = Lower prices
   • Large circles = Further from schools
   • Small circles = Closer to schools
